In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
#import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics

In [3]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [4]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "macro_f1_score":   "$MACRO_F1",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision", "macro_f1_score" ] + params
        return df[cols]
    return df

In [5]:
def get_window_classifier_results(prefix):
    collections = "WINDOW_CLASSIFIER_BR,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS".split(",")
    dfs = []
    for c in collections:
        col = prefix + c
        print(col)
        df = dict(get_df_sorted_by_f1score(col).iloc[0,:])
        df["Collection_" + prefix[:-1]] = col.replace(prefix,"")
        dfs.append(df)
    return pd.DataFrame(dfs).sort_values("micro_f1_score", ascending=False)

In [6]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

## Generate Per Class Metrics

In [72]:
def get_algo_name(coll):
    if "WINDOW" in coll:
        return "Window-Based Tagger"
    if "CRF" in coll:
        return "CRF"
    if "HMM" in coll:
        return "HMM"
    if "PERCEPTRON" in coll:
        return "Structured Perceptron"
    if "RNN" in coll:
        return "Bidirectional RNN"
    
def get_dataset_name(coll):
    if "CB" in coll:
        return "Coral Bleaching"
    if "SC" in coll:
        return "Skin Cancer"
    raise Exception("Unable to determined dataset name in method - get_dataset_name")

In [8]:
from collections import namedtuple

Metric = namedtuple("Metric", "code f1 prec rec")
Metric(code="50", f1=0.99, prec=0.98, rec=0.97)

Metric(code='50', f1=0.99, prec=0.98, rec=0.97)

In [55]:
MICRO_F1 = "MICRO_F1"
MACRO_F1 = "MACRO_F1"

def sort_key_by_code(code):
    if "b" in code:
        code = code.replace("b", ".1")
    if code == MICRO_F1:
        return 9999999
    return float(code)

def sort_key(metric):
    code = metric.code
    return sort_key_by_code(metric.code)

def get_metrics_by_code(coll):
    metrics_by_code = {}
    count = 0
    for row in db[coll].find({}):
        count += 1
        if count > 1:
            raise Exception("Multiple rows found in get_metrics_by_code for collection: " + coll)
        for k in row.keys():
            if k[0].isdigit():
                code, f1, prec, rec = k, row[k]["f1_score"], row[k]["precision"], row[k]["recall"]
                metric = Metric(code=code, f1=f1, prec=prec, rec=rec)
                metrics_by_code[code] = metric
    return metrics_by_code

def compute_metrics_by_class(coll):
    print("code\t\tf1\trec\tprec")
    count = 0
    for row in db[coll].find({}):
        count += 1
        if count > 1:
            raise Exception("More than one row found for collection: %s method - compute_metrics_by_class" % (coll))        
        keys = []                
        metrics = []
        f1s, recs, precs = [],[],[]
        macro_f1 = -1
        for k in row.keys():
            if k[0].isdigit() or k == MICRO_F1:
                keys.append(k)
                code, f1, prec, rec = k, row[k]["f1_score"], row[k]["precision"], row[k]["recall"]
                metric = Metric(code=code, f1=f1, prec=prec, rec=rec)
                metrics.append(metric)                
                # Only append metrics if a code (to ensure macro f1 is computed correctly)
                if k[0].isdigit():
                    f1s.append(f1)
                    recs.append(rec)
                    precs.append(prec)
                
            if k == MACRO_F1:
                macro_f1 = row[k]
        
        for metric in sorted(metrics, key = sort_key):
            code = metric.code
            if code == MICRO_F1:
                print()
                code = "MICRO"
            print("{code}\t{f1:.4f}\t{rec:.4f}\t{prec:.4f}".format(code=code.ljust(10), f1=metric.f1, prec=metric.prec, rec=metric.rec))

        macro_rec  = np.mean(recs)
        macro_prec = np.mean(precs)
        macro_f1_calc = (2 * (macro_rec * macro_prec)) / (macro_rec + macro_prec)
        
        print("{code}\t{f1:.4f}\t{rec:.4f}\t{prec:.4f}".format(
            code="MACRO".ljust(10), 
            f1=macro_f1, 
            prec=macro_rec, 
            rec=macro_prec))

        # Spot Check Macro F1
        #print("{code}\t{f1:.4f}\t{rec:.4f}\t{prec:.4f}".format(
        #    code="MACRO_Calc".ljust(10), 
        #   f1=macro_f1_calc, 
        #    prec=macro_rec, 
        #    rec=macro_prec))
        
        print("\n" + ",".join(sorted(keys)))
        print("")        

### CB

In [56]:
for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):

    #collection = collection.replace("CB","SC")
    print(get_algo_name(collection))
    compute_metrics_by_class(collection)
    print("")

Window-Based Tagger
code		f1	rec	prec
1         	0.8260	0.7960	0.8584
2         	0.7399	0.6871	0.8016
3         	0.8274	0.7814	0.8792
4         	0.8318	0.8193	0.8447
5         	0.4490	0.3113	0.8049
5b        	0.0303	0.0227	0.0455
6         	0.8357	0.8018	0.8725
7         	0.8381	0.7598	0.9344
11        	0.8995	0.8775	0.9227
12        	0.8629	0.7798	0.9659
13        	0.7340	0.6921	0.7814
14        	0.7484	0.7437	0.7532
50        	0.9041	0.8803	0.9293

MICRO     	0.8415	0.8023	0.8849
MACRO     	0.7400	0.7995	0.6887

1,11,12,13,14,2,3,4,5,50,5b,6,7,MICRO_F1


CRF
code		f1	rec	prec
1         	0.8365	0.8007	0.8756
2         	0.7622	0.7415	0.7842
3         	0.8197	0.7737	0.8715
4         	0.7934	0.7922	0.7946
5         	0.3188	0.2075	0.6875
5b        	0.0000	0.0000	0.0000
6         	0.8337	0.7793	0.8964
7         	0.8263	0.7394	0.9364
11        	0.9268	0.9314	0.9223
12        	0.8629	0.7798	0.9659
13        	0.7291	0.6794	0.7868
14        	0.7002	0.6835	0.7176
50        	0.9000	0.8839	0.9167

### SC

In [22]:
for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):
    collection = collection.replace("CB","SC")
    print(get_algo_name(collection))
    compute_metrics_by_class(collection)
    print("")

Window-Based Tagger
code		f1	rec	prec
1         	0.8262	0.7888	0.8673
2         	0.8525	0.8445	0.8606
3         	0.8349	0.8185	0.8520
4         	0.7330	0.6786	0.7969
5         	0.8518	0.8341	0.8702
6         	0.6788	0.5525	0.8799
11        	0.6214	0.4848	0.8649
12        	0.5294	0.4091	0.7500
50        	0.8355	0.8246	0.8467

1,11,12,2,3,4,5,50,6


CRF
code		f1	rec	prec
1         	0.8011	0.7419	0.8706
2         	0.8347	0.8434	0.8262
3         	0.7916	0.7700	0.8144
4         	0.7312	0.6673	0.8087
5         	0.8509	0.8249	0.8785
6         	0.7061	0.5869	0.8861
11        	0.6604	0.5303	0.8750
12        	0.5294	0.4091	0.7500
50        	0.8308	0.7924	0.8731

1,11,12,2,3,4,5,50,6


HMM
code		f1	rec	prec
1         	0.7121	0.7992	0.6421
2         	0.7450	0.7777	0.7150
3         	0.7247	0.8442	0.6348
4         	0.5288	0.5865	0.4815
5         	0.8018	0.8157	0.7884
6         	0.5796	0.6984	0.4953
11        	0.6066	0.5606	0.6607
12        	0.4027	0.3409	0.4918
50        	0.6413	0.6768	0.6094

1,11,

### Generate Latex Tables

In [151]:
header = """
\\begin{table}[H]
\\small
\\caption{Test Data Metrics for the ALGO on the DATASET Dataset}
\\center{\\begin{tabular}{l c c c}
\\toprule
\\textbf{Code} & \\textbf{$F_1$} & \\textbf{Recall} & \\textbf{Precision}\\\\
\\midrule"""
# Have to print {{ and }} to escape the curly braces in format string
# Add in the format strings later
header = header \
.replace("{","{{").replace("}","}}") \
.replace("ALGO","{algo}") \
.replace("DATASET","{dataset}")

footer = """\\bottomrule
\\end{tabular}}
\\label{table:rq1_ALGO_DATASET}
\\end{table}"""
# Have to print {{ and }} to escape the curly braces in format string
footer = footer \
.replace("{","{{").replace("}","}}") \
.replace("ALGO", "{algo_for_table_lbl_metrics}") \
.replace("DATASET", "{dataset_for_table_lbl}")

In [152]:
def compute_metrics_by_class_LATEX(coll):
    algo_name = get_algo_name(coll)
    dataset = get_dataset_name(coll)
    
    print(header.format(algo=algo_name, dataset=dataset))
    count = 0
    for row in db[coll].find({}):
        count += 1
        if count > 1:
            raise Exception("More than one row found for collection: %s method - compute_metrics_by_class" % (coll))        
        keys = []                
        metrics = []
        f1s, recs, precs = [],[],[]
        macro_f1 = -1
        for k in row.keys():
            if k[0].isdigit() or k == MICRO_F1:
                keys.append(k)
                code, f1, prec, rec = k, row[k]["f1_score"], row[k]["precision"], row[k]["recall"]
                metric = Metric(code=code, f1=f1, prec=prec, rec=rec)
                metrics.append(metric)                
                # Only append metrics if a code (to ensure macro f1 is computed correctly)
                if k[0].isdigit():
                    f1s.append(f1)
                    recs.append(rec)
                    precs.append(prec)
                
            if k == MACRO_F1:
                macro_f1 = row[k]
        
        row_template = """{code} &	{f1:.3f}			&	{rec:.3f}			&	{prec:.3f} \\\\ """
        for metric in sorted(metrics, key = sort_key):
            code = metric.code
            if code == MICRO_F1:
                print("\midrule")
                code = "Micro"
            print(row_template.format(code=code, f1=metric.f1, prec=metric.prec, rec=metric.rec))

        macro_rec  = np.mean(recs)
        macro_prec = np.mean(precs)
        macro_f1_calc = (2 * (macro_rec * macro_prec)) / (macro_rec + macro_prec)       
        
        print(row_template.format(code="Macro", f1=macro_f1_calc, prec=macro_prec, rec=macro_rec)) 
        
        # Footer
        #   \label{{table:rq1_{{algo_for_table_lbl_metrics}}_{{dataset_for_table_lbl}}}}
        dataset_abbrev = "".join([word[0] for word in dataset.lower().split(" ")])
        print(footer.format(
                algo_for_table_lbl_metrics=algo_name.lower().strip().replace(" ","_").replace("-","_"),
                dataset_for_table_lbl=dataset_abbrev
        ))
    
        #print("*" * 60)
        #print("\n" + ",".join(sorted(keys)))    

In [153]:
#CB
print("\\section{Coral Bleaching}")
for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):
    compute_metrics_by_class_LATEX(collection)
    print("")

#SC 
print("\\section{Skin Cancer}")
for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):
    collection = collection.replace("CB","SC")
    compute_metrics_by_class_LATEX(collection)
    print("")

\section{Coral Bleaching}

\begin{table}[H]
\small
\caption{Test Data Metrics for the Window-Based Tagger on the Coral Bleaching Dataset}
\center{\begin{tabular}{l c c c}
\toprule
\textbf{Code} & \textbf{$F_1$} & \textbf{Recall} & \textbf{Precision}\\
\midrule
1 &	0.826			&	0.796			&	0.858 \\ 
2 &	0.740			&	0.687			&	0.802 \\ 
3 &	0.827			&	0.781			&	0.879 \\ 
4 &	0.832			&	0.819			&	0.845 \\ 
5 &	0.449			&	0.311			&	0.805 \\ 
5b &	0.030			&	0.023			&	0.045 \\ 
6 &	0.836			&	0.802			&	0.873 \\ 
7 &	0.838			&	0.760			&	0.934 \\ 
11 &	0.899			&	0.877			&	0.923 \\ 
12 &	0.863			&	0.780			&	0.966 \\ 
13 &	0.734			&	0.692			&	0.781 \\ 
14 &	0.748			&	0.744			&	0.753 \\ 
50 &	0.904			&	0.880			&	0.929 \\ 
\midrule
Micro &	0.842			&	0.802			&	0.885 \\ 
Macro &	0.740			&	0.689			&	0.800 \\ 
\bottomrule
\end{tabular}}
\label{table:rq1_window_based_tagger_cb}
\end{table}


\begin{table}[H]
\small
\caption{Test Data Metrics for the CRF on the Coral Bleaching Dataset}
\center{\begin{tabular}{l c c

# What is the Best Algorithm Per Class, By Metric?

In [25]:
cb_codes = "1,2,3,4,5,5b,6,7,11,12,13,14,50".split(",")
sc_codes = "1,2,3,4,5,6,11,12,50".split(",")
len(cb_codes), len(sc_codes)

(13, 9)

In [26]:
Metric4Code = namedtuple("Metric4Code", "algo f1 rec prec")

def sort_by_f1_desc(metric):
    return -metric.f1

def sort_by_rec_desc(metric):
    return -metric.rec

def sort_by_prec_desc(metric):
    return -metric.prec

### CB

In [14]:
ALGO_NAME_WIDTH = len("Structured Perceptron") + 1

for code in cb_codes:
    metrics_for_code = []
    for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):
        m4code = get_metrics_by_code(collection)[code]   
        algo_name = get_algo_name(collection)
        metrics_for_code.append(Metric4Code(algo=algo_name, f1=m4code.f1, rec=m4code.rec, prec=m4code.prec))
    
    best_f1 = sorted(metrics_for_code, key=sort_by_f1_desc)[0]
    best_rec = sorted(metrics_for_code, key=sort_by_rec_desc)[0]
    best_prec = sorted(metrics_for_code, key=sort_by_prec_desc)[0]
    print("Code: {code}\tF1:{f1}\tRec:{rec}\tPrec:{prec}".format(
        code=code.ljust(3), 
        f1=best_f1.algo.ljust(ALGO_NAME_WIDTH), 
        rec=best_rec.algo.ljust(ALGO_NAME_WIDTH), 
        prec=best_prec.algo.ljust(ALGO_NAME_WIDTH)
    ))

Code: 1  	F1:Structured Perceptron 	Rec:Bidirectional RNN     	Prec:CRF                   
Code: 2  	F1:Structured Perceptron 	Rec:HMM                   	Prec:Structured Perceptron 
Code: 3  	F1:Window-Based Tagger   	Rec:Window-Based Tagger   	Prec:Bidirectional RNN     
Code: 4  	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:Window-Based Tagger   
Code: 5  	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:Window-Based Tagger   
Code: 5b 	F1:Bidirectional RNN     	Rec:HMM                   	Prec:Bidirectional RNN     
Code: 6  	F1:Window-Based Tagger   	Rec:Window-Based Tagger   	Prec:CRF                   
Code: 7  	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:Structured Perceptron 
Code: 11 	F1:CRF                   	Rec:CRF                   	Prec:Window-Based Tagger   
Code: 12 	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:Window-Based Tagger   
Code: 13 	F1:Bidirectional RNN     	Rec:HMM                   	Prec:Structured Perceptron 

In [ ]:
for code in cb_codes:
    metrics_for_code = []
    for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):
        #print(collection)
        m4code = get_metrics_by_code(collection)[code]   
        algo_name = get_algo_name(collection)
        metrics_for_code.append(Metric4Code(algo=algo_name, f1=m4code.f1, rec=m4code.rec, prec=m4code.prec))
    
    best_f1 = sorted(metrics_for_code, key=sort_by_f1_desc)[0]
    best_rec = sorted(metrics_for_code, key=sort_by_rec_desc)[0]
    best_prec = sorted(metrics_for_code, key=sort_by_prec_desc)[0]
    print("Code: {code}\tF1:{f1}\tRec:{rec}\tPrec:{prec}".format(
        code=code.ljust(3), 
        f1=best_f1.algo.ljust(ALGO_NAME_WIDTH), 
        rec=best_rec.algo.ljust(ALGO_NAME_WIDTH), 
        prec=best_prec.algo.ljust(ALGO_NAME_WIDTH)
    ))

### SC

In [27]:
for code in sc_codes:
    metrics_for_code = []
    for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):
        collection = collection.replace("CB_", "SC_")
        m4code = get_metrics_by_code(collection)[code]   
        algo_name = get_algo_name(collection)
        metrics_for_code.append(Metric4Code(algo=algo_name, f1=m4code.f1, rec=m4code.rec, prec=m4code.prec))
    
    best_f1 = sorted(metrics_for_code, key=sort_by_f1_desc)[0]
    best_rec = sorted(metrics_for_code, key=sort_by_rec_desc)[0]
    best_prec = sorted(metrics_for_code, key=sort_by_prec_desc)[0]
    print("Code: {code}\tF1:{f1}\tRec:{rec}\tPrec:{prec}".format(
        code=code.ljust(3), 
        f1=best_f1.algo.ljust(ALGO_NAME_WIDTH), 
        rec=best_rec.algo.ljust(ALGO_NAME_WIDTH), 
        prec=best_prec.algo.ljust(ALGO_NAME_WIDTH)
    ))

Code: 1  	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:Bidirectional RNN     
Code: 2  	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:Window-Based Tagger   
Code: 3  	F1:Bidirectional RNN     	Rec:HMM                   	Prec:Bidirectional RNN     
Code: 4  	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:Bidirectional RNN     
Code: 5  	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:CRF                   
Code: 6  	F1:Bidirectional RNN     	Rec:HMM                   	Prec:CRF                   
Code: 11 	F1:Bidirectional RNN     	Rec:HMM                   	Prec:Bidirectional RNN     
Code: 12 	F1:Structured Perceptron 	Rec:Structured Perceptron 	Prec:Bidirectional RNN     
Code: 50 	F1:Bidirectional RNN     	Rec:Bidirectional RNN     	Prec:Bidirectional RNN     
